## Procesado de Ficheros IV
---

### 4. Representando texto
---
Al igual que otros, un campo de la AI donde el _deep learning_ está teniendo un fuerte impacto es el del **_natural_language_processing_** (NLP), especialmente mediante el empleo de **_recurrent neural netoworks_** (RNNs), que consumen una combinación de nuevas entradas y la salida de modelos previos. Más recientemente, una nueva clase de redes denominadas _transformers_ con modos más flexibles de incorporar información pasada ha tenido un gran impacto. Actualmente, en el area del NLP, en lugar de diseñar sofisticados _pipelines_ de varias etapas y que incluían reglas gramaticales codificadas, se diseñan redes que se entrenan sobre un _corpus_ dejando que esas reglas "emerjan" de los datos.

En este caso, nuestro objetivo será transformar un texto en algo que pueda procesar una red neuronal, un tensor de números.



#### 4.1 Convirtiendo texto a números
---
Hay dos niveles en los que una red puede trabajar con un texto, a nivel de carácter o a nivel de palabra. En todo caso, elijamos la opción que elijamos, aplicaremos codificación **on-hot** sobre la entidad correspondiente.

Vamos a empezar con codificación a nivel de caracter.

Primeramente, deberemos hacernos con cierta cantidad de texto. Tenemos múltiples opciones: [Proyecto Gutenberg](http://www.gutenberg.org), el _corpus_ de Wikipedia (1.9 miles de millones de palabras, 4.4 millones de artículos), [English Corpora](http://www.english-corpora.org),...

Para nuestra práctica vamos a utilizar el libro _Pride and Prejudice_ de Jane Austen descargado de la web del Proyecto Gutenberg

In [13]:
with open("data/text/1342-0.txt", encoding='utf-8') as f:
    text = f.read()
text[:100]

'\ufeff\nThe Project Gutenberg EBook of Pride and Prejudice, by Jane Austen\n\nThis eBook is for the use of a'

##### 4.1.1 Codificación one-hot de caracteres
---
Previamente al procesado del texto, deberemos hacer varias asunciones y preparaciones, con objeto de facilitar las tareas posteriores. Una sería limitar la codificación texto-binario que vamos a procesar (por ejemplo, ASCII). Otra, por ejemplo, sería convertir todo el texto a minúsculas, eliminar puntuaciones, números,...

El siguiente paso sería recorrer todo el texto y proporcionar una codificación _on-hot_ para cada carácter, consistente en un vector de longitud el _corpus_ de caracteres con todos los valores a 0 salvo para el índice del carácter que estamos codificacndo, que valdrá 1.

A modo de ejemplo, vamos a extraer una línea del texto para hacer nuestras pruebas

In [20]:
lines = text.split('\n')
line = lines[339]
line

'      “Impossible, Mr. Bennet, impossible, when I am not acquainted'

Ahora, vamos a crear un tensor que pueda almacenar todos los caracteres (codificados one-hot) para la línea

In [21]:
import torch

letter_t = torch.zeros(len(line), 128)  # ASCII limit
letter_t.shape

torch.Size([67, 128])

Nuestro tensor tendrá una fila por cada carácter de la frase, y 128 columnas correspondientes a la codificación _one-hot_ del mismo. El número de columnas viene determinado por el conjunto de los diferentes caracteres del _corpus_ del texto (en nuestro caso, sólo ASCII).

Ahora, lo único que tendremos que hacer, será, para cada caracter del texto, colocar un 1 en el índice que le corresponda (según su codificación ASCII)

In [24]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_t[i, letter_index] = 1
letter_t

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

##### 4.1.2 Codificación one-hot de palabras
---
La codificación _one-hot_ de palabras se puede hacer de la misma manera que de caracteres, estableciendo un vocabulario y la codificación correspondiente de las palabras del texto. Debido a que el tamaño de un vocabulario es extramadamente amplio, este esquema no es muy práctico. Un modelo más eficiente se basa, como veremos posteriormente, en el empleo de _embeddings_.

Vamos a procesar la linea anterior y codificarla por palabras. Primeramente, preprocesaremos la frase para elminar signos de puntuación y convertir a minúsculas.

In [32]:
def clean_words(input_str):
    punctuation = '.,;:"!?“”_-'
    word_list = input_str.lower().replace('\n', " ").split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('      “Impossible, Mr. Bennet, impossible, when I am not acquainted',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted'])

Vamos a crear ahora un _mapping_ de palabras a índices para nuestro encoding (el vocabulario). Básicamente, construiremos un diccionario a partir del texto, usando cada palabra como clave y un índice como valor


In [33]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word:i for (i,word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(7278, 3383)

Vamos a ver cómo resultaría la codificcación _one-hot_ de nuestra línea de texto

In [36]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i,word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i, word_index] = 1
    print(f"{i:2} {word_index:4} {word}")

print(word_t.shape)

 0 3383 impossible
 1 4298 mr
 2  796 bennet
 3 3383 impossible
 4 7071 when
 5 3304 i
 6  397 am
 7 4425 not
 8  220 acquainted
torch.Size([9, 7278])


La elección entre codificación basada en caracteres o palabras supone asumir ciertas consideraciones. Por un lado, existen muchos menos caracteres que palabras, por lo que necesitamos muchas menos clases para representarlos. Por otro, las palabras contienen mucho más significado que los caracteres individuales, por los que codificar palabras es mucho más informativo. En la práctica, se suelen adoptar caminos intermedios, como el _byte pairs encoding method_: se empieza con un diccionario de letras individuales para ir añadiendo los grupos de caracteres que aparecen con mayor frecuencia hasta alcanzar el tamaño de diccionario predefinido.

<br>

![](data/image-lect/encoding-words.png)

<br>

### 4.2 _Text embeddings_
---
La codificación _one-hot_ es muy úyil para representar datos nominales o categóricos, pero empieza a fallar cuando el número de items a codificar es excesivamente grande, como las palabras de un _corpus_ lingüístico. En el ejemplo anterior, para un único libro, tenemos sobre 7000 items!. Una codificación _one-hot_ de tales dimensiones es claramente excesiva.

Podemos hacer cierto preprocesado eliminando duplicados, condensado tiempos verbales,... pero sería un trabajo ímprobo. Además, nuevas palabras supondrían añadirlas al encoding y entrenar de nuevo.

El **_embedding_** consiste en mapear las palabras en un vector de números floatantes de tamaño predeterminado (por ejemplo, 100). En principio, podríamos iterar sobre nuestro vocabulario y generar 100 números aleatorios para el vector correspondiente, pero omitiría cualquier tipo de distancia basada en significado o contenido entre las palabras. Lo ideal sería que palabras empleadas en contextos similares fueran mapeadas a regiones próximas del _embedding_.

Por ejemplo, si decidiéramos construir manualmente nuestro _embedding_, podríamos simplemente empezar con un mapeo básico a nombres y adjetivos (dos ejes). Así, podríamos distribuir las distintas palabras sobre este espacio 2D como muestra la siguiente figura:

<br>

![](data/image-lect/embedding.png)

<br>

Podemos ver como palabras como las palabras que referencian colores se alinean sobre el eje de los adjetivos mientras que otras, como _dog_, _fruit_ o _flower_, sobre el eje de los nombres. En torno a ellas, situaríamos variantes de perros, frutas o flores más próximas a aquellos nombres o adjetivos relacionados.

Sin duda, una clasificación manual de este tipo sería imprácticable para un _corpus_ grande y con múltiples ejes. Sin embargo, _embeddings_ (de entre 100 y 1000 dimensiones) pueden construirse de forma automática mediante el empleo de redes neuronales que se encargarán de crear _clusters_ de palabras a partir del contexto y proximidad a otras palabras en el texto.




